# TP3: Exploring other denoiser architecture


In this lesson we will introduce the *layer inspectors*, tools for inspecting the intermediate state of a network and use it to study DnCNN. Then we will apply it analyze a multiscale denoising network called U-Net. Finally we will study a network architectures inspired on variational methods.

We will cover the following topics:
* Peeking inside the DnCNN network with layer inspectors  
* U-Net architecture - with and without skip connections
* What can we see by applying the layer inspector to U-net?
* Strangling networks

There are **5 questions** in the notebook and corresponding text areas to fill-in the answers. 


#### Instructions
To solve this TP, answer the questions below. Then export the notebook with the answers using  the menu option **File->Download as->HTML**. Send the resulting *html* file by mail to [facciolo@cmla.ens-cachan.fr](mailto:facciolo@cmla.ens-cachan.fr) with subject "Report tp1 of SURNAME, Name", by 30/11/2018.  You will receive an acknowledgement of receipt.

In [ ]:
# Setup code for the notebook

# Execute code 'cells' like this by clicking on the 'Run' 
# button or by pressing [shift] + [Enter].

# This cell only imports some python packages that will be
# used below. It doesn't generate any output. Something similar 
# applies to the next two or three cells. They only define 
# functions that are used later.


# This notebook can also run on colab (https://colab.research.google.com/)
# The following lines install the necessary packages in the colab environment
try:
    from google.colab import files
    !pip install torch==0.4.1
    !pip install torchvision
    !pip install Pillow==4.0.0
    !pip install scikit-image
    !pip install hdf5storage

    !rm -fr MVAdenoising2018
    !git clone  https://github.com/gfacciol/MVAdenoising2018
    !cp -r MVAdenoising2018/* .

except ImportError:
    # %matplotlib notebook
    pass


# These are all the includes used through the notebook
import numpy as np
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
import vistools          # image visualization toolbox
from   skimage import io # read and write images
from   vistools import unzip


# global variable for setting the torch.load    map_location
if torch.cuda.is_available():
    loadmap = {'cuda:0': 'gpu', 'location': 'gpu'}
else:
    loadmap = {'cuda:0': 'cpu', 'location': 'cpu'}
    

#%matplotlib notebook
# Autoreload external python modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

# Peeking inside the DnCNN network with layer inspectors

We will introduce layer inspectors to see the intermediate results for the hidden layers of a network. We define a layer inspector as the simplest affine transformation (convolutional) that takes a hidden layer and produces an output image. In DnCNN, each layer inspectors are $1\times 1$ convolutional layers that take the features at a given layer and produce a single image. These inspectors have to be trained. We will trained them *after* DnCNN was trained.

Below we define an inspector module `DnCNNInspector`, which receives a trained DnCNN network  $\mathcal F$ as input. For each layer of DnCNN we add an inpector layer, which we'll denote by $\mathcal I^l_{\theta}$. The inspector is applied to the output of layer $l$ of the denoising network $\mathcal F$. We will denote that output $\mathcal F^l(u)$, where $u$ is the input image.

Note that we only train the parameters of the inspectors. We prevent training the parameters of the input network, by setting their `require_grad` field to false with the following lines:

``` python
        # we don't want to train the dncnn parameters
        for p in self.dncnn.parameters():
            p.requires_grad = False
```

In [ ]:
from models import DnCNN

class DnCNNInspector(nn.Module):
    def __init__(self, dncnn):
        """
        Args: 
            - dncnn: a dncnn network
        """
        super(DnCNNInspector, self).__init__()
            
        # retrieve parameters of dncnn network
        self.num_layers   = len(dncnn.layers)
        self.num_features = dncnn.layers[0].conv.weight.shape[0]
        self.kernel_size  = dncnn.layers[0].conv.weight.shape[2]
        self.out_channels = dncnn.layers[-1].weight.shape[0]
        self.in_channels  = dncnn.layers[0].conv.weight.shape[1]
        self.residual     = dncnn.residual
        
        import copy
        self.dncnn = copy.deepcopy(dncnn)
        
        # we don't want to train the dncnn parameters
        for p in self.dncnn.parameters():
            p.requires_grad = False
        
        # define inspector layers
        self.inspectors = []
        for i in range(self.num_layers-1):
            self.inspectors.append(nn.Conv2d(self.num_features,
                                             self.out_channels,
                                             1)) # <-- 1x1 kernel
            name = 'inspector%d' % i
            self.register_parameter(name + 'weight', self.inspectors[i].weight)
            self.register_parameter(name + 'bias'  , self.inspectors[i].bias)

   
    def forward(self, x):
        """
        """
        
        # force eval mode in dncnn network
        self.dncnn.eval()

        outputs = []
        out = x
        for i in range(self.num_layers-1):
            # apply layer of dncnn
            out = self.dncnn.layers[i](out)
            
            # apply inspector
            if self.residual:
                outputs.append(x - self.inspectors[i](out))
            else:
                outputs.append(self.inspectors[i](out))

        # returns a tensor having all the produced outputs as channels
        # the number of channels is (num_layers-1)*ch, where ch is the 
        # number of channels of x
        return torch.cat(outputs,-3)

To train the inspectors we need a custom loss (because the inspector network produces multiple output images). The following block defines our `multiOutputMSELoss`. Our loss will be the MSE between the images produced by each inspector and the clean image:
$$R^{\text{emp}}(\theta) = \frac 1{L-1}\sum_i \sum_{l=1}^{l = L-1} \|\mathcal I^l_\theta(\mathcal F^l(u)) - \widetilde u\|^2.$$
The parameters $\theta$ here are the weights and biases of each inspector.

In [ ]:
# we define this loss, so that we can train all inspectors in parallel
def multiOutputMSELoss(x, y):
    """
    Computes the average MSE loss multiple outputs x_1, ...., x_n with respect
    to a single target y.
    
    Args:
        - x: outputs, 4D tensor of size [b, n*ch, w, h] where b is the batch size, n is
             the number of ouputs, ch is the number of channels of each output and wxh
             their spatial dimension
        - y: target, 4D tensor of size [b, ch, w, h]
    
    Returns:
        - loss: average MSE loss, i.e. 1/n * \sum_i MSELoss(x_i,y)
    """
    
    if x.shape[-3] % y.shape[-3] != 0:
        print('multiOutputMSELoss: x num. of channels is not a multiple of y num. of channels')
    n = x.shape[-3]//y.shape[-3]
    
    return ((x - torch.cat([y]*n,1))**2).mean()

We are now ready to train our inspectors. In the following blocks we train and apply the instector network to peack inside DnCNN by displaying the intermediate results thoughout the network for image.

In [ ]:
# train layer inspectors by minimizing the multi output MSE loss

from denoising_dataloaders import train_val_denoising_dataloaders
from training import trainmodel
from models import DnCNN, DnCNN_pretrained

sigma=30


# DO NOT set this flag to true if running on the server.
# The results are already pre-computed.
training=False

if training:

    # data
    trainloader, validationloader = train_val_denoising_dataloaders(
                                              './datasets/Train400/', 
                                               noise_sigma=sigma, crop_size=40, 
                                               train_batch_size=128)

    # load a dncnn network, and build an inspector for it
    #dncnn = torch.load('trainings/tiny_DnCNN_mse_2000.pt', map_location=loadmap))[0]
    dncnn = DnCNN_pretrained(sigma)
    dncnn_ins = DnCNNInspector(dncnn)

    # run the training loop
    dncnn_ins, losst, lossv, = trainmodel(dncnn_ins, multiOutputMSELoss, trainloader, validationloader, 
                                         num_epochs=100, save_every=100, loss_every=10,  
                                         learning_rate=0.01, weight_decay=0.00001,
                                         filename='pre-trained-tp3/layer_inspector_DnCNN_')

else:
    dncnn_ins, _, losst, lossv = torch.load('pre-trained-tp3/layer_inspector_DnCNN_0100.pt', map_location=loadmap)

# plot loss
plt.semilogy(lossv, label='val')
plt.semilogy(losst, label='train')
plt.legend(); plt.xlabel('epoch'); plt.ylabel('loss');

In [ ]:
from skimage import io
from denoising_helpers import PSNR
from vistools import unzip

# test it with a noisy image
sigma=30
im_clean = io.imread('datasets/BSD68/test002.png', dtype='float32') 
im_noisy = im_clean + np.random.normal(0, sigma, im_clean.shape)

# put the image in the range [0,1] and add noise
im_noisy = im_noisy.astype('float32') / 255.

# create inpector from existing dncnn
dncnn = DnCNN_pretrained(sigma)

# torch data type
dtype = torch.FloatTensor
if torch.cuda.is_available():
    # run on GPU
    dncnn_ins = dncnn_ins.cuda()
    dncnn = dncnn.cuda()
    dtype = torch.cuda.FloatTensor

# set denoising network in evaluation (inference) mode
dncnn_ins.eval()
dncnn.eval()

# apply inspector network
with torch.no_grad(): # tell pytorch that we don't need gradients
    img = dtype(im_noisy[np.newaxis,np.newaxis,:,:]) # convert to tensor
    out  = dncnn(img).cpu()
    outs = dncnn_ins(img).cpu()

out_ims = list()
for i in range(outs.shape[1]):
    out_ims.append( (outs[0,i,:,:]*255, 'inspector %d - PSNR = %f' %(i, PSNR(outs[0,i,:,:]*255, im_clean))) )    

out_ims.append( (out*255, 'output - PSNR = %f' %(PSNR(out*255, im_clean))))
out_ims.append( (im_noisy*255, 'noisy - PSNR = %f' %(PSNR(im_noisy*255, im_clean))))
    
# show as a gallery
vistools.display_gallery(unzip(out_ims,0), unzip(out_ims,1))

**Question 1.**  Comment on the intermediate results on the network. Compare the outputs of the layers with the last ones. Does the PSNR increase through the network?

ANSWER TO QUESTION 1

# U-Net

We will now study the U-Net instroduced in the context of image segmentation in

    O. Ronneberger, P. Fischer, T. Brox, "U-Net: Convolutional Networks for Biomedical Image Segmentation," Medical Image Computing and Computer-Assisted Intervention (MICCAI), Springer, LNCS, Vol.9351: 234--241, 2015

and used in the context of image restoration in: [C. Chen, Q. Chen, J. Xu, V. Koltun, "Learning to See in the Dark,"](https://arxiv.org/abs/1805.01934)
 
The U-Net has a U-shaped architecture as illustrated below:

<img width=500 src="https://raw.githubusercontent.com/gfacciol/MVAdenoising2018/master/models/unet.png"/>

The definition of our U-Net is `models.UNet`. The following code trains it (or loads a pre-trained one).

In [ ]:
from torch import nn
from models import UNet
from denoising_dataloaders import train_val_denoising_dataloaders
from training import trainmodel


sigma=30

# DO NOT set this flag to true if running on the server.
# The results are already pre-computed.
training=False


# choose loss
#loss_name = 'l1'
loss_name = 'l2'

if training:
    
    # data
    trainloader, valloader = train_val_denoising_dataloaders(
                                              './datasets/Train400/', 
                                               noise_sigma=sigma, crop_size=128, 
                                               train_batch_size=45 )

    # network model
    denoiser = UNet(1,1)    

    # run the training loop using chosen loss
    loss_fn = nn.L1Loss() if loss_name == 'l1' else nn.MSELoss()
    denoiser, losst, lossv, = trainmodel(denoiser, loss_fn, trainloader, valloader, 
                                         num_epochs=2000, save_every=500, loss_every=10,  
                                         learning_rate=0.001, weight_decay=0.00001,
                                         filename='pre-trained-tp3/Unet_%s_' % (loss_name))

In [ ]:
from skimage import io
import torch
from models import UNet
from denoising_helpers import test_denoiser

denoiser, _, losst, lossv = torch.load('pre-trained-tp3/Unet_l2_2000.pt', map_location=loadmap)[0:4]

plt.semilogy(lossv, '.-', label='net1 val')
plt.semilogy(losst, '.-', label='net1 train')
plt.legend(); plt.xlabel('epoch'); plt.ylabel('loss');
plt.show()

denoiser.cpu()

img_clean = io.imread('datasets/BSD68/test002.png', dtype='float32') 

_ = test_denoiser(denoiser, img_clean, sigma=30, show=True)

**Question 2.** 
In the block below, write the code needed for comparing the performance of U-Net with DnCNN, and FFDNet. Compare speed, PSNR, and visual quality. 

**Tips**
* Follow the comments below and re-use code from other blocks (from previous TPs)
* To measure the execution time you can use the command :  ```%timeit  output = command(input)```

In [ ]:
###############################
###   ANSWER TO QUESTION 2  ###
###############################

from skimage import io
import torch
from models import UNet, DnCNN_pretrained, DnCNN, FFDNet_pretrained_grayscale, FFDNet
from denoising_helpers import test_denoiser

sigma = 30

# load image
im_clean = io.imread('datasets/BSD68/test002.png', dtype='float32') 
im_noisy = im_clean + np.random.normal(0, sigma, im_clean.shape)

# list of labelled results to display as gallery
outputs = list()


# let's load U-Net first ###############################################

# load network
denoiser = torch.load( # TODO file with pretrained U-Net # )[0]
denoiser.cpu()
    
# denoise image with loaded net
out = test_denoiser(denoiser, im_noisy, None, has_noise=True)[0] 
outputs.append( (out, 'U-Net - PSNR = %f' %(PSNR(out, im_clean))) )    

# TODO now test DnCNN_pretrained #################################

# TODO finally FFDNet_pretrained_grayscale #################################

    
# show as a gallery
vistools.display_gallery(unzip(outputs,0), unzip(outputs,1))

# Layer inspector for U-Net

U-Net downscales the image through the network. To inpect these downscaled layers, we need our inpectors to up-scale them. One way of doing that is by transposing a convolutional layer with a stride. A convolutional layer with a stride of 2, produces an output which is half the size of the input. Thus by transposing it, we produce an output which has twice the input size.

In the next block we define our `UNetInspector` module. It attaches one inspector layer for each scale. In the scales that have skip connections, we attach the inspector *before* upscaling, as shown in the diagram: 

<img width=500 src="models/unetInspector.png"/>

**Attention the following code only works with images with size multiple of 16!**

In [ ]:
from models import UNet

class UNetInspector(nn.Module):
    def __init__(self, unet, trainall=False):
        """
        Args: 
            - unet: a unet network
            - trainall: train the unet? (default False)
        """
        super(UNetInspector, self).__init__()
            
        # retrieve parameters of unet network ?? any
        self.n_channels   = unet.outc.conv.weight.shape[0] 
        self.trainall  = trainall # train the UNET 

        import copy
        self.unet = copy.deepcopy(unet)
        
        # we don't want to train the unet parameters
        if not trainall:
            for p in self.unet.parameters():
                p.requires_grad = False
        
        # define inspector layers
        self.inspectors = []
        

        # then add the other inspectors from higher to lower resolutions
        ss = [2,4,8,16]  # strides
        kk = [4,8,16,32] # kernel sizes
        pp = [1,2,4,8]   # padding
        ff = [64, 128, 256, 512]   # features
        for i in range(4):            
#            nfeatures = min(pow(2,i)*64,512)  # DEBUG
#           print(nfeatures, pow(2,i))         # DEBUG
            self.inspectors.append( nn.ConvTranspose2d(
                                      ff[i], self.n_channels, 
                                      kk[i], stride=ss[i], 
                                      padding=pp[i] ) )
            
            name = 'inspector%d' % i
            self.register_parameter(name + 'weight', self.inspectors[i].weight)
            self.register_parameter(name + 'bias'  , self.inspectors[i].bias)

            
            
        i = 4
        # first add sanity check inspector at the end of the network
        self.inspectors.append( nn.Conv2d(
                                  64, self.n_channels, 
                                  3, padding=1 ) )
        
        name = 'inspector%d' % i
        self.register_parameter(name + 'weight', self.inspectors[i].weight)
        self.register_parameter(name + 'bias'  , self.inspectors[i].bias)  
        
                
        
    def forward(self, x):
        """
        """
        
        # force eval mode in dncnn network
        if not self.trainall:
            self.unet.eval()

        outputs = []        
                
        x1 = self.unet.inc(x)
        x2 = self.unet.down1(x1)
        x3 = self.unet.down2(x2)
        x4 = self.unet.down3(x3)
        
        x5 = self.unet.down4(x4)
        outputs.append( self.inspectors[3](x5) ) 

        x = self.unet.up1(x5, x4)
        outputs.append( self.inspectors[2](x) ) 

        x = self.unet.up2(x, x3)
        outputs.append( self.inspectors[1](x) ) 

        x = self.unet.up3(x, x2)        
        outputs.append( self.inspectors[0](x) ) 

        x = self.unet.up4(x, x1)
        outputs.append( self.inspectors[4](x) ) 
        
        x = self.unet.outc(x)
        
        outputs.append( x ) 

        # returns a tensor having all the produced outputs as channels
        # the number of channels is (num_layers-1)*ch, where ch is the 
        # number of channels of x
        return torch.cat(outputs,-3)

We now train and apply the inspector networks.

In [ ]:
# train layer inspectors by minimizing the multi output MSE loss

from denoising_dataloaders import train_val_denoising_dataloaders
from training import trainmodel
from models import DnCNN

sigma=30


# DO NOT set this flag to true if running on the server.
# The results are already pre-computed.
training=False


if training:
    # data
    trainloader, valloader = train_val_denoising_dataloaders(
                                              './datasets/Train400/', 
                                               noise_sigma=sigma, crop_size=128, 
                                               train_batch_size=32)

    # load a dncnn network, and build an inspector for it
    denoiser, _, trainloss, valloss = torch.load('pre-trained-tp3/Unet_l2_2000.pt', map_location=loadmap)[0:4]
    denoiser_ins = UNetInspector(denoiser)

    # run the training loop
    denoiser_ins, losst, lossv, = trainmodel(denoiser_ins, multiOutputMSELoss, trainloader, valloader, 
                                             num_epochs=200, save_every=100, loss_every=10,  
                                             learning_rate=0.01, weight_decay=0.0,
                                             filename='pre-trained-tp3/layer_inspector_Unet_')

else:
    denoiser_ins, _, losst, lossv = torch.load('pre-trained-tp3/layer_inspector_Unet_0200.pt', map_location=loadmap)

# plot loss
plt.semilogy(lossv, label='val')
plt.semilogy(losst, label='train')
plt.legend(); plt.xlabel('epoch'); plt.ylabel('loss');

In [ ]:
from models import DnCNN_pretrained
from skimage import io
from denoising_helpers import PSNR
from vistools import unzip

# test it with a noisy image
sigma=30

im_clean = io.imread('datasets/BSD68/test002.png', dtype='float32')[0:480,0:320]  #<<<<< HERE WE FOCE THE SHAPE

# put the image in the range [0,1] and add noise
im_noisy = im_clean + np.random.normal(0, sigma, im_clean.shape)
im_noisy = im_noisy.astype('float32') / 255.

# create inpector from existing dncnn
denoiser_ins = torch.load('pre-trained-tp3/layer_inspector_Unet_0200.pt', map_location=loadmap)[0]

# torch data type
dtype = torch.FloatTensor
if torch.cuda.is_available():
    # run on GPU
    denoiser_ins = denoiser_ins.cuda()
    dtype = torch.cuda.FloatTensor

# set denoising network in evaluation (inference) mode
denoiser_ins.eval()

# apply inspector network
with torch.no_grad(): # tell pytorch that we don't need gradients
    img = dtype(im_noisy[np.newaxis,np.newaxis,:,:]) # convert to tensor
    out = denoiser_ins(img).cpu()

outs = []
for i in range(out.shape[1]):
    outs.append( (out[0,i,:,:]*255, 'inspector %d - %f (dB)' % (i, PSNR(out[0,i,:,:]*255, im_clean)) ) )

outs.append( (im_noisy*255, 'noisy - %f (dB)' % (PSNR(im_noisy, im_clean)) ) )

vistools.display_gallery(unzip(outs,0), unzip(outs,1))

**Question 3.** What do you observe in the above experiment. Is this what you expected? Comment on that.

ANSWER TO QUESTION 3.

# U-Net without skip connections

To understand the importance of skip connections, we will train a network without them:

<img width=500 src="models/unetX.png"/>

In [ ]:
from torch import nn
from models import UNet
from denoising_dataloaders import train_val_denoising_dataloaders
from training import trainmodel



sigma=30

# DO NOT set this flag to true if running on the server.
# The results are already pre-computed.
training=False


# choose loss
#loss_name = 'l1'
loss_name = 'l2'

if training:
    
    # data
    trainloader, validationloader = train_val_denoising_dataloaders(
                                              './datasets/Train400/', 
                                               noise_sigma=sigma, crop_size=128, 
                                               train_batch_size=45 )

    # network model without skip connections
    denoiser =  UNet(1,1, skipc=False)

    # run the training loop using chosen loss
    loss_fn = nn.L1Loss() if loss_name == 'l1' else nn.MSELoss()
    denoiser, losst, lossv, = trainmodel(denoiser, loss_fn, trainloader, validationloader, 
                                         num_epochs=2000, save_every=250, loss_every=10,  
                                         learning_rate=0.001, weight_decay=0.00001,
                                         filename='pre-trained-tp3/Unet_noskip_%s_' % (loss_name))

In [ ]:
from skimage import io
import torch
from models import UNet
from denoising_helpers import test_denoiser


denoiser, _, losst, lossv = torch.load('pre-trained-tp3/Unet_noskip_l2_2000.pt', map_location=loadmap)[0:4]
plt.semilogy(losst[0:])
plt.semilogy(lossv[0:])

plt.show()



import vistools


img_clean = io.imread('datasets/BSD68/test002.png', dtype='float32') 
#img_clean = io.imread('datasets/LIVE1/bikes.bmp', dtype='float32')[:,:,1]

_=test_denoiser(denoiser, img_clean, sigma=30, show=True)

This result shouldn't be surprising. Let's look at the inspectors for the U-Net without skip connections.

In [ ]:
# train layer inspectors by minimizing the multi output MSE loss

from denoising_dataloaders import train_val_denoising_dataloaders
from training import trainmodel
from models import UNet

sigma=30


# DO NOT set this flag to true if running on the server.
# The results are already pre-computed.
training=False


if training:
    # data
    trainloader, valloader = train_val_denoising_dataloaders(
                                              './datasets/Train400/', 
                                               noise_sigma=sigma, crop_size=128, 
                                               train_batch_size=32)

    # load a dncnn network, and build an inspector for it
    denoiser, _, trainloss, valloss = torch.load('pre-trained-tp3/Unet_noskip_l2_2000.pt', map_location=loadmap)[0:4]
    denoiser_ins = UNetInspector(denoiser)

    # run the training loop
    denoiser_ins, losst, lossv, = trainmodel(denoiser_ins, multiOutputMSELoss, trainloader, valloader, 
                                             num_epochs=200, save_every=100, loss_every=10,  
                                             learning_rate=0.01, weight_decay=0.0,
                                             filename='pre-trained-tp3/layer_inspector_Unet_noskip_')

else:
    denoiser_ins, _, losst, lossv = torch.load('pre-trained-tp3/layer_inspector_Unet_noskip_0200.pt', map_location=loadmap)

# plot loss
plt.semilogy(lossv, label='val')
plt.semilogy(losst, label='train')
plt.legend(); plt.xlabel('epoch'); plt.ylabel('loss');

In [ ]:
from models import DnCNN_pretrained
from skimage import io
from denoising_helpers import PSNR
from vistools import unzip

# test it with a noisy image
sigma=30
im_clean = io.imread('datasets/BSD68/test002.png', dtype='float32')[0:480,0:320]

# put the image in the range [0,1] and add noise
im_noisy = im_clean + np.random.normal(0, sigma, im_clean.shape)
im_noisy = im_noisy.astype('float32') / 255.

# create inpector from existing dncnn
denoiser_ins = torch.load('pre-trained-tp3/layer_inspector_Unet_noskip_0200.pt', map_location=loadmap)[0]

# torch data type
dtype = torch.FloatTensor
if torch.cuda.is_available():
    # run on GPU
    denoiser_ins = denoiser_ins.cuda()
    dtype = torch.cuda.FloatTensor

# set denoising network in evaluation (inference) mode
denoiser_ins.eval()

# apply inspector network
with torch.no_grad(): # tell pytorch that we don't need gradients
    img = dtype(im_noisy[np.newaxis,np.newaxis,:,:]) # convert to tensor
    out = denoiser_ins(img).cpu()

outs = []
for i in range(out.shape[1]):
    outs.append( (out[0,i,:,:]*255, 'inspector %d - %f (dB)' % (i, PSNR(out[0,i,:,:]*255, im_clean)) ) )

outs.append( (im_noisy*255, 'noisy - %f (dB)' % (PSNR(im_noisy, im_clean)) ) )

vistools.display_gallery(unzip(outs,0), unzip(outs,1))

**Question 4.** 
Compare these results with the outputs of inpectors for the U-Net (with the skip connections). How do the skip connections alter the information encoded at each scale.

ANSWER TO QUESTION 4.

# Strangled DnCNN

We will now return to DnCNN (our default denoising network) to do an experiment. We will include, in our tiny DnCNN a strangling hidden layer with with one channel.

In [ ]:
# definition of DnCNN module with added strangled layers.

class CONV_BN_RELU(nn.Module):
    '''
    PyTorch Module grouping together a 2D CONV, BatchNorm and ReLU layers.
    This will simplify the definition of the DnCNN network.
    '''

    def __init__(self, in_channels=128, out_channels=128, kernel_size=7, 
                 stride=1, padding=3):
        '''
        Constructor

        Args:
            - in_channels: number of input channels from preceding layer
            - out_channels: number of output channels
            - kernel_size: size of conv. kernel
            - stride: stride of convolutions
            - padding: number of zero padding

        Return: initialized module
        '''
        super(__class__, self).__init__()

        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size, 
                              stride=stride, padding=padding)
        self.bn   = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)
        
    def forward(self, x):
        '''
        Applies the layer forward to input x
        '''
        out = self.conv(x)
        out = self.bn(out)
        out = self.relu(out)
        
        return(out)



class StrangledDnCNN(nn.Module):
    '''
    PyTorch module for the DnCNN network.
    '''

    def __init__(self, in_channels=1, out_channels=1, num_layers=17, 
                 features=64, kernel_size=3, residual=True,
                 strangling_layers = None):
        '''
        Constructor

        Args:
            - in_channels: input image channels (default 1)
            - out_channels: output image channels (default 1)
            - num_layers: number of non-strangling layers (default 17)
            - num_features: number of hidden features (default 64)
            - kernel_size: size of conv. kernel (default 3)
            - residual: use residual learning (default True)
            - strangling_layers: indices of the layers where strangling is added

        Return: initialized network
        '''
        super(__class__, self).__init__()
        
        if strangling_layers is None:
            strangling_layers = (num_layers-2)//2
        
        strangling_mask = np.zeros(num_layers)
        strangling_mask[strangling_layers] = 1
        
        self.residual = residual
        
        # a list for the layers
        self.layers = []
        
        # first layer
        self.layers.append(CONV_BN_RELU(in_channels=in_channels,
                                        out_channels=features,
                                        kernel_size=kernel_size,
                                        stride=1, padding=kernel_size//2))
        # half of intermediate layers
        for l in range(num_layers-2):
            self.layers.append(CONV_BN_RELU(in_channels=features,
                                            out_channels=features,
                                            kernel_size=kernel_size,
                                            stride=1, padding=kernel_size//2))
            
            if strangling_mask[l+1]:
                # strangling layers
                self.layers.append(CONV_BN_RELU(in_channels=features,
                                                out_channels=in_channels,
                                                kernel_size=kernel_size,
                                                stride=1, padding=kernel_size//2))

                self.layers.append(CONV_BN_RELU(in_channels=in_channels,
                                                out_channels=features,
                                                kernel_size=kernel_size,
                                                stride=1, padding=kernel_size//2))

        # last layer 
        self.layers.append(nn.Conv2d(in_channels=features,
                                     out_channels=out_channels,
                                     kernel_size=kernel_size,
                                     stride=1, padding=kernel_size//2))
        # chain the layers
        self.dncnn = nn.Sequential(*self.layers)

        
    def forward(self, x):
        ''' Forward operation of the network on input x.'''
        out = self.dncnn(x)
        
        if self.residual: # residual learning
            out = x - out 
        
        return(out)

In [ ]:
from denoising_dataloaders import train_val_denoising_dataloaders
from training import trainmodel
from models import DnCNN
from torch import nn


sigma=30

# DO NOT set this flag to true if running on the server.
# The results are already pre-computed.
training=False

if training:
    # data
    trainloader, valloader = train_val_denoising_dataloaders(
                                              './datasets/Train400/', 
                                               noise_sigma=sigma, crop_size=40, 
                                               train_batch_size=32 )

    # network model
    denoiser = StrangledDnCNN(in_channels=1, out_channels=1, 
                              num_layers=7,
                              features=13,
                              kernel_size=3, 
                              residual=True,
                              strangling_layers=[3]) # Strangled1
#                               strangling_layers=[2,4]) # Strangled2


    # loss
    loss = nn.MSELoss()

    # run the training loop
    denoiser, losst, lossv, = trainmodel(denoiser, loss, trainloader, valloader, 
                                        num_epochs=2000, save_every=1000, loss_every=200,  
                                        learning_rate=0.01, weight_decay=0.00001,
                                        filename='pre-trained-tp3/tiny_Strangled1_DnCNN_')

else:
    denoiser, _, losst, lossv = torch.load('pre-trained-tp3/tiny_Strangled1_DnCNN_2000.pt', map_location=loadmap)

# plot loss
plt.semilogy(lossv, label='val')
plt.semilogy(losst, label='train')
plt.legend(); plt.show()

In [ ]:
# Compare the denoising results

from skimage import io
from denoising_helpers import test_denoiser, PSNR
from vistools import unzip

# load denoising nets

# load an image
img_clean = io.imread('datasets/BSD68/test002.png', dtype='float32')
img_noisy = img_clean + np.random.normal(0, sigma, img_clean.shape)

outs = list()

net = torch.load('pre-trained-tp2/tiny_DnCNN_2000.pt', map_location=loadmap)[0]
out = test_denoiser(net, img_noisy, sigma, has_noise=True)[0]
outs.append( (out, 'tiny DnCNN - %f (dB)' % (PSNR(out, img_clean)) ) ) 

net = torch.load('pre-trained-tp3/tiny_Strangled1_DnCNN_2000.pt', map_location=loadmap)[0]
out = test_denoiser(net, img_noisy, sigma, has_noise=True)[0]
outs.append( (out, 'tiny DnCNN 1 strangling layer - %f (dB)' % (PSNR(out, img_clean)) ) ) 

net = torch.load('pre-trained-tp3/tiny_Strangled2_DnCNN_2000.pt', map_location=loadmap)[0]
out = test_denoiser(net, img_noisy, sigma, has_noise=True)[0]
outs.append( (out, 'tiny DnCNN 2 stragling layers - %f (dB)' % (PSNR(out, img_clean)) ) ) 

outs.append( (np.array(img_clean).clip(0,255), 'clean'))
outs.append( (np.array(img_noisy).clip(0,255), 'noisy'))

vistools.display_gallery(unzip(outs,0), unzip(outs,1))

**Question 6.** What's the effect of the strangling layers in the networks performance? What would expect about the performance of the *trainable inference* networks that we saw in the theory.

ANSWER TO QUESTION 6.

---------------------------
[//]: # (© 2018 Gabriele Facciolo and Pablo Arias)
[//]: # (<div style="text-align:center; font-size:75%;"> Copyright © 2018 Gabriele Facciolo and Pablo Arias. All rights reserved.</div> )